<a href="https://colab.research.google.com/github/bartoszlibner/usda/blob/main/usda_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
!pip install PyDrive

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)

Next cell contains libraries required to authenticate to GoogleDrive. In GoogleDrive a database in excel sheet form of all the USDA ingredients

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded_foundation_food = drive.CreateFile({'id':"1qBqmL9i9ff9UL5n2Tp8oWYE1XDiHRYE0"})   
downloaded_foundation_food.GetContentFile('foundation_food.csv')        

downloaded_sr_legacy = drive.CreateFile({'id':"1sqhYFFxdR-uannIidxkIjj9RqMbjempe"})   
downloaded_sr_legacy.GetContentFile('sr_legacy_food.csv')        

downloaded_sr_legacy = drive.CreateFile({'id':"122upIOEuxK0rtciVyzCk7WEIBx4UBW5v"})   
downloaded_sr_legacy.GetContentFile('sr_legacy_food_portions.csv')        

downloaded_sr_legacy = drive.CreateFile({'id':"1iG2rZ1U2_Rgma6ex6ZqsK0VZh_5HPsKt"})   
downloaded_sr_legacy.GetContentFile('sr_legacy_food_nutrient.csv')        


downloaded_sr_legacy = drive.CreateFile({'id':"1G1nUfAtMNcJuSFRHtun2ZMoFX5q9HXsc"})   
downloaded_sr_legacy.GetContentFile('nutrient.csv')        

In [ ]:
data = pd.read_csv('foundation_food.csv')
sr_legacy_data = pd.read_csv('sr_legacy_food.csv')
sr_legacy_data_portions = pd.read_csv('sr_legacy_food_portions.csv')
sr_legacy_data_nutrients = pd.read_csv('sr_legacy_food_nutrient.csv')
df_nutrient_descriptions = pd.read_csv('nutrient.csv')

In [ ]:
df = data.copy()

There are differences between what USDA is saying in their "Download field description file" and what we actually have in this table. There should be couple of fields more that should have more data, but we can still work with it IMO

# exploring the amount of unique data types and numbers connexted to them

In [ ]:
df['data_type'].unique()

array(['sample_food', 'market_acquisition', 'sub_sample_food',
       'foundation_food', 'agricultural_acquisition'], dtype=object)

In [ ]:
mark_safo_noduplicates = df[df['data_type'] == 'sample_food'].drop_duplicates(subset='description')
mark_safo_noduplicates

In [ ]:
mark_acui_noduplicates = df[df['data_type'] == 'market_acquisition'].drop_duplicates(subset='description')
len(mark_acui_noduplicates)

3519

In [ ]:
df['data_type'].where(df['data_type'] == 'agricultural_acquisition').count()

810

In [ ]:
# it seems that foundation food is very limited and not all the ingredients end up here. Even here we have some data from restaurants
df_foundation_df = df[df['data_type'] == 'foundation_food'].drop_duplicates(subset='description')
df_foundation_df[df_foundation_df['description'].str.contains('flour', case=False)]

In [ ]:
df['data_type'].where(df['data_type'] == 'sub_sample_food').count()

12400

In [ ]:
market_acquisition_df = df[df['data_type'] == 'market_acquisition'].copy()
#market_acquisition_df['description'].unique().tolist();
market_acquisition_df[market_acquisition_df['description'].str.contains('pork', case=False)]

In [ ]:
market_acquisition_df = df[df['data_type'] == 'agricultural_acquisition'].copy()
market_acquisition_df['description'].unique().tolist()

#Exploring the description column

In [ ]:
#new_df = df['description'].drop_duplicates(subset('description'))
#new_df
df_sample_food_only = df.loc[df['data_type'] == 'sample_food'].copy()
df_sample_food_only_count = df.loc[df['data_type'] == 'sample_food'].count()
print("Number of unique food samples")
print(df_sample_food_only_count)

print("Number of unique food samples with unique descriptions")
df_sample_food_only['description'].unique().tolist();

Number of unique food samples
fdc_id              1828
data_type           1828
description         1828
food_category_id    1813
publication_date    1828
dtype: int64
Number of unique food samples with unique descriptions


In [ ]:
#how to search for particulat string, good source is here https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285
new_df = df[df['description'].str.contains('^chicken', case=False, regex=True) & df['data_type'].str.contains('^sample_food') & df['description'].str.contains("\d", regex=True)].copy()
new_df.drop_duplicates(subset=['description'], inplace=True)
new_df;

# Exploring the different columns in database 

The goal is to have a json formatted database of ingredients, that isn't cluttered by information users don't need

In [ ]:
df['data_type'].unique()

array(['sample_food', 'market_acquisition', 'sub_sample_food',
       'foundation_food', 'agricultural_acquisition'], dtype=object)

In [ ]:
#confirming that all the food have at least sample_food and foundational_food
df[(df['data_type']=='sample_food') | (df['data_type'] == 'foundation_food')]

unique_agri = df[(df['data_type']=='agricultural_acquisition')]
unique_agri_descriptions = unique_agri['description'].unique().tolist()

unique_market = df[(df['data_type']=='market_acquisition')]
unique_market_descriptions = unique_market['description'].unique().tolist()

unique_foundation_food = df[(df['data_type']=='foundation_food')]
unique_foundation_food_descriptions = unique_foundation_food['description'].unique().tolist()

unique_sample_food = df[(df['data_type']=='sample_food')]
unique_sample_food_descriptions = unique_sample_food['description'].unique().tolist()
#df[df['data_type'] == 'foundation_food'];
len(np.setdiff1d(unique_agri_descriptions, unique_market_descriptions))

805

In [ ]:
df_without_sub_sample_food = df[~(df['data_type'] == 'sub_sample_food')].copy()
df_without_sub_sample_food_no_dup = df_without_sub_sample_food.drop_duplicates(subset=['data_type','description'])
df_without_sub_sample_food_no_dup.sort_values(by=['description'])
df_without_sub_sample_food_no_dup['food_category_id'].unique()

array([16.,  1., 13., 11.,  2.,  7., 12.,  6.,  9., 18.,  4.,  5., 15.,
       19., 25., 10., nan, 20.])

new_df = df[df['data_type'].str.contains('^sample_food')].copy()
new_df.drop_duplicates(subset=['description'], inplace=True)
new_df;

## Conclusions from Foundation Food DB 
From this particular DB, we should be only using foundation_foods, as they are averaged versions of all the other samples and market acquisitions. Market acquisitions are sometimes put together to make one sample_foods and then based on that sample_food foundation_food is made. I didn't find ingredients so far that wouldn't fit that. 

You can experiment yourself using the code below. It prints very nicely the given category.



In [ ]:
# we can simply use foundation in this case. Each of the sample_foods 
df_cleaned_cat1 = df_without_sub_sample_food_no_dup[df_without_sub_sample_food_no_dup['food_category_id'] == 4]
#df_cleaned_cat1_foundation_only = df_cleaned_cat1[df_cleaned_cat1['data_type'] == 'foundation_food']
#df_cleaned_cat1_foundation_only.count()
df_cleaned_cat1

,fdc_id,data_type,description,food_category_id,publication_date
9709,330416,sample_food,"Oil, coconut, LOU ANA (NC) CY1211H",4.0,2019-04-01
9710,330417,market_acquisition,"Oil, coconut, LOU ANA (NC1) NFY120PCL",4.0,2019-04-01
9711,330418,market_acquisition,"Oil, coconut, LOU ANA (NC1) NFY120PCK",4.0,2019-04-01
9714,330421,sample_food,"Oil, coconut, LOU ANA (IN,AL) CY1211I",4.0,2019-04-01
9715,330422,market_acquisition,"Oil, coconut, LOU ANA (AL1) NFY120PCM",4.0,2019-04-01
9716,330423,market_acquisition,"Oil, coconut, LOU ANA (IN1) NFY120PCO",4.0,2019-04-01
9719,330426,sample_food,"Oil, coconut, LOU ANA (MI,OK) CY1211K",4.0,2019-04-01
9720,330427,market_acquisition,"Oil, coconut, LOU ANA (MI1) NFY120PCQ",4.0,2019-04-01
9721,330428,market_acquisition,"Oil, coconut, LOU ANA (OK1) NFY120PCS",4.0,2019-04-01
9722,330429,market_acquisition,"Oil, coconut, LOU ANA (OK1) NFY120PCT",4.0,2019-04-01


# Exploring SR Legacy DB

In [ ]:
sr_data = sr_legacy_data.copy()

So, it turned out that all the all caps words (apart from USDA) were were actually names of brands. Excluding them 

In [ ]:
print(sr_data.columns)
sr_data.sort_values(by='description')
usda_only = sr_data[sr_data['description'].str.contains('USDA')]
sr_data_cleaned = sr_data[~(sr_data['description'].str.contains("[A-Z]{2,}", regex=True))].sort_values(by='description')
sr_data_cleaned.head()
for_compare = sr_data[(sr_data['description'].str.contains("[A-Z]{2,}", regex=True))].sort_values(by='description')
#all good, but exlude USDA

Index(['fdc_id', 'data_type', 'description', 'food_category_id',
       'publication_date'],
      dtype='object')


# Starting to transform Dataframe to final result 



In [ ]:
!pip install --upgrade gupload

How I currently imagine product model

{
* 'fdc_id': 1888544,
* 'description': 'Cookshire toast',
* 'portion': [
 * {
   * 'seq_num': 1,
   * 'amount': 0.5,
   * 'modifier': cup
   * 'gram_weight': 114,
 * },
 * {
   * 'seq_num':2,
   * 'amount': 0.5,
   * 'modifier': 'fl oz',
   * 'gram_weight': 30.2,
 * }
* ]
* 'nutrient': [
  * {
    * 'nutrient_id': 1003,
    * 'nutrient_amount': 5.88,
  * }
  * {
    * 'nutrient_id': 1002,
    * 'nutrient_amount': 6.33,
  * }
* ]

}



Taking original dataframes and merging them to have a good ground

In [ ]:
merged_portion = pd.merge(sr_data_cleaned, sr_legacy_data_portions, how='inner', on='fdc_id')
merged_portion.drop(columns=['publication_date', 'food_category_id', 'id', 'data_type', 'measure_unit_id', 'portion_description', 'data_points', 'footnote', 'min_year_acquired'], inplace=True)
merged_portion['type'] = 'portion'
merged_portion = merged_portion[['fdc_id', 'description', 'type', 'seq_num', 'amount', 'modifier', 'gram_weight']]
merged_portion.head(5)
len(merged_portion[merged_portion['fdc_id'] == 171687])

2

In [ ]:
# solution based on https://stackoverflow.com/questions/52923685/convert-pandas-multiindex-series-to-json-python
merged_portion_json = (merged_portion.groupby(['fdc_id','description', 'type'])['seq_num', 'amount', 'modifier', 'gram_weight']
       .apply(lambda x: x.to_dict(orient='records'))
       .reset_index(name='portion')
       #.groupby(['fdc_id', 'description', 'food_category_id', 'type'])['data']
       #.apply(lambda x: x.set_index(['type'])['data'].to_dict('split'))
       #.to_json(orient='records')
       )

#for group, frame in merged_json:
#  print(frame.columns)
#  print(frame)
#print(merged_portion_json[:6000])
merged_portion_json.head(50)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,fdc_id,description,type,portion
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'serving', 'gram_weight': 34.0}]"
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refrigerated dough",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'serving 1 roll with icing', 'gram_weight': 44.0}]"
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coating for Pork, dry",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'serving', 'gram_weight': 28.0}]"
3,167515,"George Weston Bakeries, Thomas English Muffins",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'serving', 'gram_weight': 57.0}]"
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'waffle, square', 'gram_weight': 39.0}, {'seq_num': 2, 'amount': 1.0, 'modifier': 'waffle, round', 'gram_weight': 38.0}]"
5,167517,"Waffle, buttermilk, frozen, ready-to-heat, toasted",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'oz', 'gram_weight': 28.0}, {'seq_num': 2, 'amount': 1.0, 'modifier': 'waffle round (4"" dia)', 'gram_weight': 33.0}]"
6,167518,"Waffle, buttermilk, frozen, ready-to-heat, microwaved",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'waffle', 'gram_weight': 35.0}]"
7,167519,"Waffle, plain, frozen, ready-to-heat, microwave",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'waffle, round (4""dia)', 'gram_weight': 32.0}]"
8,167520,"Pie Crust, Cookie-type, Graham Cracker, Ready Crust",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'oz', 'gram_weight': 28.35}, {'seq_num': 2, 'amount': 1.0, 'modifier': 'crust', 'gram_weight': 183.0}]"
9,167521,"Pie Crust, Cookie-type, Chocolate, Ready Crust",portion,"[{'seq_num': 1, 'amount': 1.0, 'modifier': 'crust', 'gram_weight': 182.0}]"


# PnL and Tax experiment

Experimenting to find out good way of formatting Dataframes

In [ ]:
pnl = pd.DataFrame(index=['Company A', 'Company B', 'Company B'], columns=['Product Name', 'Price'], data={'Product Name':['Orange', 'Apple', 'Grapes'],'Price':['3000', '2000', '1000']})
pnl

,Product Name,Price
Company A,Orange,3000
Company B,Apple,2000
Company B,Grapes,1000


In [ ]:
tax = pd.DataFrame(index=['Company A', 'Company B', 'Company B'], columns=['Product Name', 'Price'], data={'Product Name':['Orange', 'Apple', 'Grapes'],'Price':['100', '100', '10']})
tax

,Product Name,Price
Company A,Orange,100
Company B,Apple,100
Company B,Grapes,10


In [ ]:
df1 = pd.concat([pnl, tax], keys=('PnL','Tax')).reset_index()
df1.columns = ['type','company','productName','price']
print (df1)

  type    company productName price
0  PnL  Company A      Orange  3000
1  PnL  Company B       Apple  2000
2  PnL  Company B      Grapes  1000
3  Tax  Company A      Orange   100
4  Tax  Company B       Apple   100
5  Tax  Company B      Grapes    10


In [ ]:
d = (df1.groupby(['type','company'])['productName','price']
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name='data')
       .groupby('type')['company','data']
       .apply(lambda x: x.set_index('company')['data'].to_dict())
       .to_json()
       )

print(d[:1000])

#for group, frame in d:
#  print(frame)

{"PnL":{"Company A":[{"productName":"Orange","price":"3000"}],"Company B":[{"productName":"Apple","price":"2000"},{"productName":"Grapes","price":"1000"}]},"Tax":{"Company A":[{"productName":"Orange","price":"100"}],"Company B":[{"productName":"Apple","price":"100"},{"productName":"Grapes","price":"10"}]}}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


# Introducing food_nutrient to our DB

In [ ]:
df_nutrients = sr_legacy_data_nutrients.copy()
df_nutrients.drop(columns=['id', 'min', 'max', 'median', 'derivation_id', 'data_points', 'footnote', 'min_year_acquired'], inplace=True)
df_nutrients.rename(columns={'amount': 'nutrient_amount'}, inplace=True)
len(df_nutrients[df_nutrients['fdc_id'] == 171687])

68

# getting description and food_category_id and type into nutrients df

In cell below there is an error. It seems that each portion has it's own amount of nutrients, which is not correct according to the documentation created couple of cells higher. Resolve tomorrow

In [ ]:
nutrient = pd.merge(df_nutrients,merged_portion_json[['fdc_id', 'description']], how='left', on='fdc_id')
nutrient = nutrient[['fdc_id','description' , 'nutrient_id', 'nutrient_amount']]
#nutrient.head(50)
nutrient[nutrient['description'] == 'Acerola juice, raw']

,fdc_id,description,nutrient_id,nutrient_amount
347554,171687,"Acerola juice, raw",1062,96.000
347555,171687,"Acerola juice, raw",1186,0.000
347556,171687,"Acerola juice, raw",1259,0.000
347557,171687,"Acerola juice, raw",1260,0.000
347558,171687,"Acerola juice, raw",1262,0.000
347559,171687,"Acerola juice, raw",1265,0.048
347560,171687,"Acerola juice, raw",1266,0.016
347561,171687,"Acerola juice, raw",1268,0.081
347562,171687,"Acerola juice, raw",1269,0.046
347563,171687,"Acerola juice, raw",1270,0.044


In [ ]:
#df_nutrient_descriptions.drop(columns=['nutrient_nbr', 'rank'], inplace=True)

In [ ]:
merged_nutrient_description = pd.merge(nutrient, df_nutrient_descriptions, how='left', left_on='nutrient_id', right_on='id')
#merged_nutrient_description.head()
merged_nutrient_description.drop(columns='id', inplace=True)
merged_nutrient_description_final = merged_nutrient_description[['fdc_id', 'description', 'nutrient_id', 'name', 'nutrient_amount', 'unit_name']].copy()
merged_nutrient_description_final.rename(columns={'name':'nutrient_name', 'unit_name':'nutrient_unit_name'}, inplace=True)
merged_nutrient_description_final
len(merged_nutrient_description_final[merged_nutrient_description_final['description'] == 'Acerola juice, raw'])

68

# preprocessing nutrients to a dict

In [ ]:
nutrients_dict = (merged_nutrient_description_final.groupby(['fdc_id', 'description'])['nutrient_id', 'nutrient_name', 'nutrient_amount', 'nutrient_unit_name']
     .apply(lambda x: x.to_dict(orient='records'))
     .reset_index(name='nutrient')
     #.to_json(orient='records')
)

#d[:6000]
#merged_json['fdc_id'].index
#series.compare(merged_json['fdc_id'].sort_values())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


# putting all data together into one df

In [ ]:
#concatenated_nutrients = pd.concat([merged_portion, nutrient])
#concatenated_nutrients.sort_values(by='fdc_id', inplace=True)
#print(concatenated_nutrients.columns)

merged_portion_nutrient = pd.merge(nutrients_dict, merged_portion_json, how='right', on=['fdc_id', 'description'])
merged_portion_nutrient.drop(columns='type',inplace=True)
merged_portion_nutrient = merged_portion_nutrient[['fdc_id', 'description', 'portion','nutrient']]

#to json file
res = merged_portion_nutrient.to_json('ingredients.json', orient='records')

#print here
res = merged_portion_nutrient.to_json(orient='records')
print(res[:20000])


[{"fdc_id":167512,"description":"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough","portion":[{"seq_num":1,"amount":1.0,"modifier":"serving","gram_weight":34.0}],"nutrient":[{"nutrient_id":1003,"nutrient_name":"Protein","nutrient_amount":5.88,"nutrient_unit_name":"G"},{"nutrient_id":1007,"nutrient_name":"Ash","nutrient_amount":3.5,"nutrient_unit_name":"G"},{"nutrient_id":1062,"nutrient_name":"Energy","nutrient_amount":1286.0,"nutrient_unit_name":"kJ"},{"nutrient_id":1079,"nutrient_name":"Fiber, total dietary","nutrient_amount":1.2,"nutrient_unit_name":"G"},{"nutrient_id":1089,"nutrient_name":"Iron, Fe","nutrient_amount":2.12,"nutrient_unit_name":"MG"},{"nutrient_id":1093,"nutrient_name":"Sodium, Na","nutrient_amount":1059.0,"nutrient_unit_name":"MG"},{"nutrient_id":1253,"nutrient_name":"Cholesterol","nutrient_amount":0.0,"nutrient_unit_name":"MG"},{"nutrient_id":1257,"nutrient_name":"Fatty acids, total trans","nutrient_amount":4.412,"nutrient_unit_name"

# exporting to json file 

In [ ]:
!gupload --to '1sSrmiwT6GX3EylUouR_lakdR3x1qX1xi' ingredients.json 

Uploading file: ingredients.json


# Product search demo

In [ ]:
#merged_portion_nutrient[merged_portion_nutrient['description'].str.contains('wheat flour',case=False)]
pd.set_option('display.max_colwidth', None)
dfp = merged_portion_nutrient[merged_portion_nutrient['fdc_id'] == 167705]
dfp['nutrient'].to_csv('167705.csv')


In [ ]:
!gupload --to '1sSrmiwT6GX3EylUouR_lakdR3x1qX1xi' '167705.csv' 

Uploading file: 167705.csv
